In [1]:
import numpy as np
import numpyro
import numpyro.distributions as dist
from jax import random
from numpyro.infer import MCMC, HMC,NUTS
import fitsio
import matplotlib as mpl
from multiprocessing import Pool
from scipy.stats import dirichlet
from astropy.table import Table
from astropy.io import fits
import fitsio
from astropy.cosmology import Planck18
import jax.numpy as jnp
import matplotlib.pyplot as plt
from numpyro import handlers 
from jax import random, jit
import jax
from fitting import abs_mag_to_L,get_lfpars_shen20,T0,T1,T2,get_lfpars,get_phis,integral,integral_Lmin_Lmax
from jax import random, vmap
from jax.scipy.stats import norm
import numpyro.distributions as dist
from jax.scipy.special import gammaln
from astropy.cosmology import WMAP9 as cosmo
from scipy.optimize import curve_fit, brentq
import fitting
from scipy.optimize import minimize

In [2]:
jax.config.update("jax_enable_x64", True)

In [3]:
k = Table.read('/pscratch/sd/z/zhaozhon/DATASET/DESI-data/k.csv',format='csv')
indice = np.isnan(k['k_rr']) | np.isinf(k['k_rr'])
#k_gg = k['k_gg'][~indice]
k_rr = k['k_rr'][~indice]

In [4]:
file = '/pscratch/sd/z/zhaozhon/DATASET/DESI-data/new_d.fits'
d = Table(fitsio.read(file))
SEL = np.where((d['Z'] > 2.3) & (d['Z'] < 2.8))[0]
TARGETID = d['TARGETID'][SEL][~indice]
#apr_mag_g = d['APP_MAG_G'][SEL][~indice]
#apr_mag_g_err = d['APP_MAGERR_G'][SEL][~indice]
apr_mag_r = d['APP_MAG_R'][SEL][~indice]
apr_mag_r_err = d['APP_MAGERR_R'][SEL][~indice]
z_desi = d['Z'][SEL][~indice]

In [5]:
d_desi = Planck18.distmod(z_desi).value

In [6]:
gamma1, gamma2, L_star, phi_star = get_lfpars_shen20(2.35)
alpha=-(gamma1+1)
beta = -(gamma2+1)
print(alpha,beta,phi_star)

-1.41011 -3.50877883045515 1.37670217962694e-05


In [7]:
TARGETID, indices = np.unique(TARGETID, return_index=True)
#apr_mag_g = apr_mag_g[indices]
#apr_mag_g_err = apr_mag_g_err[indices]
apr_mag_r = apr_mag_r[indices]
apr_mag_r_err = apr_mag_r_err[indices]
d_desi = d_desi[indices]
z_desi = z_desi[indices]
#k_gg = k_gg[indices]
k_rr = k_rr[indices]

In [8]:
# Assuming z_desi is your redshift array
filter_mask = (z_desi > 2.3) & (z_desi < 2.4)

# Apply the filter to all arrays
TARGETID = TARGETID[filter_mask]
#apr_mag_g = apr_mag_g[filter_mask]
#apr_mag_g_err = apr_mag_g_err[filter_mask]
apr_mag_r = apr_mag_r[filter_mask]
apr_mag_r_err = apr_mag_r_err[filter_mask]
d_desi = d_desi[filter_mask]
z_desi = z_desi[filter_mask]
#k_gg = k_gg[filter_mask]
k_rr = k_rr[filter_mask]

# Recalculate N_obs after filtering
N_obs = TARGETID.shape[0]
print(N_obs)

42124


In [9]:
apr_mag_r = apr_mag_r.astype(np.float64)
apr_mag_r_err = apr_mag_r_err.astype(np.float64)
k_rr = k_rr.astype(np.float64)

d_desi = d_desi.astype(np.float64)
z_desi = z_desi.astype(np.float64)

In [10]:
print(apr_mag_r.max())

23.027341842651367


In [11]:
def eff(m,b,m0):
    eff = jnp.where(m < m0, 1,jnp.where(m < 23.1,10**(-b*(m-m0)/2.5),0))
    return eff

ln_posterior = fitting.ln_posterior(eff)

def neg_likelihood(params,apr_mag,k,mu,apr_mag_err):
    m0,b,x,fraction = params
    log_likelihood = ln_posterior(m0, b, x, apr_mag, k, mu, apr_mag_err,fraction)
    #log_likelihood = ln_posterior(m0, b, x, apr_mag, k, mu, apr_mag_err,0.07977749883319056)
    return -log_likelihood

In [12]:
abs_mag_to_L(apr_mag_r.max()-k_rr.mean()-d_desi.mean()-1.74)

3.275350286280196e+45

In [15]:
initial_guess = [18.826109855329346,0.5023055585762686,0.8297,0.16]
bounds = [(0,22),(0,1),(0,2),(0.16,0.16)]

result = minimize(
    neg_likelihood, 
    initial_guess, 
    args=(apr_mag_r, k_rr, d_desi, apr_mag_r_err), 
    method='L-BFGS-B',
    bounds=bounds,
    options={'ftol': 1e-8, 'gtol': 1e-6, 'maxiter': 100000}
)

optimal_params = result.x
optimal_likelihood = -result.fun  

print(f"Optimal parameters: m0 = {optimal_params[0]}, b = {optimal_params[1]}, x = {optimal_params[2]},fraction = {optimal_params[3]}")
print(f"Maximum likelihood value: {optimal_likelihood}")

first term -60247.69384459738
Poisson term 401647.31907029636
first term -60247.69384460174
Poisson term 401647.3189725864
first term -60247.693845939546
Poisson term 401647.3193951989
first term -60247.69377351371
Poisson term 401647.319185435
first term -57393.018629029335
Poisson term -329465.73828359967
first term -57393.018629029335
Poisson term -329465.73789562326
first term -57393.018627720914
Poisson term -329465.73029661627
first term -57393.01858930534
Poisson term -329465.7378713988
first term -59610.6101099246
Poisson term 403617.9895534969
first term -59610.6101099246
Poisson term 403617.98962083017
first term -59610.61011126716
Poisson term 403617.98918673873
first term -59610.61004372292
Poisson term 403617.9894767003
first term -59547.981155621965
Poisson term 406417.94507943187
first term -59547.981155621106
Poisson term 406417.9450834304
first term -59547.9811569647
Poisson term 406417.9450576814
first term -59547.981089879395
Poisson term 406417.94507468084
first ter

In [14]:
fitting.integral_Lmin_Lmax(Lmin, jnp.inf, beta, alpha)

NameError: name 'Lmin' is not defined

In [ ]:
params = [18.613685598951363,35,18.613685598951363,0.16]
neg_likelihood(params,apr_mag_r, k_rr, d_desi, apr_mag_r_err)

In [ ]:
'''
params = [16.0259,0.45,1.85]
neg_likelihood(params,apr_mag_r,k_rr,d_desi,apr_mag_r_err)
'''

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

# Define the parameter grid for m0 and b
x_values = np.linspace(-5, 5, 10)
#m0_values = np.linspace(1.5, 2.5, 10)
b_values = np.linspace(1, 10, 10)

# Calculate likelihoods for a fixed x
#x_fixed = 1.7
m0_fixed = 18
#b_fixed = 0.5
likelihoods = np.zeros((len(x_values), len(b_values)))

for i, x in enumerate(tqdm(x_values)):
    for j, b in enumerate(b_values):
        params = [m0_fixed, b, x]
        likelihoods[i, j] = neg_likelihood(params,apr_mag_r, k_rr, d_desi, apr_mag_r_err)

# Create a heatmap
plt.figure(figsize=(8, 6))
plt.contourf(x_values, b_values, likelihoods.T, levels=50, cmap='viridis')
plt.colorbar(label='Negative Log-Likelihood')
plt.xlabel('$x$')
plt.ylabel('$b$')
plt.title('Negative Log-Likelihood Heatmap ($x$ fixed)')
plt.show()

In [ ]:
params = [18.358572258469366,0.5762614390624853,1.85]
neg_likelihood(params,apr_mag_r, k_rr, d_desi, apr_mag_r_err)